In [1]:
import json
from openai import OpenAI

In [ ]:
client = OpenAI()

def generar_historias(json_input_file, json_output_file, prompt_base):
    # Cargar el JSON
    with open(json_input_file, "r", encoding="utf-8") as f:
        issues = json.load(f)

    for issue in issues:
        title = issue.get("title", "")
        body = issue.get("body", "")
        state = issue.get("state", "")
        author = issue.get("user", "")
        created = issue.get("created_at", "")
        issue_number = issue.get("number", "")

        # Armar contexto
        context_lines = []
        context_lines.append(f"Issue #{issue_number} reportado por {author} (creado el {created}):\n")
        context_lines.append(f"Título: {title}\n")
        context_lines.append(f"Descripción:\n{body}\n")
        context_lines.append(f"Estado: {state}\n")
        context_lines.append("Comentarios:")

        # Agregar comentarios de la issue al contexto
        comments = issue.get("all_comments", [])
        if comments:
            for comment in comments:
                c_author = comment.get("user", {}).get("login", "")
                c_date = comment.get("created_at", "")
                c_body = comment.get("body", "").strip()
                context_lines.append(f"- {c_author} ({c_date}): {c_body}")
        else:
            context_lines.append("- No hay comentarios.")

        # Agregar etiquetas de la issue al contexto
        labels = issue.get("labels", [])
        if labels:
            label_names = [label.get("name", "") for label in labels]
            context_lines.append(f"Etiquetas: {', '.join(label_names)}\n")
        else:
            context_lines.append("Etiquetas: (ninguna)\n")

        # Concatenar contexto + prompt base
        full_prompt = "\n".join(context_lines) + "\n\n" + prompt_base

        print(f"Generando historia de usuario para issue #{issue_number}...")

        # Llamar al LLM
        completion = client.chat.completions.create(
           model="gpt-4o",
            messages=[
                {"role": "system", "content": "Eres un experto en redacción de historias de usuario."},
                {"role": "user", "content": full_prompt}
            ],
            temperature=0.2
        )

        respuesta = completion.choices[0].message.content.strip()

        # Guardar en el issue
        issue["historia_usuario"] = respuesta

    # Guardar JSON enriquecido
    with open(json_output_file, "w", encoding="utf-8") as f:
        json.dump(issues, f, indent=2, ensure_ascii=False)

    print(f"Historias generadas y guardadas en '{json_output_file}'.")



In [ ]:
# Ejemplo de uso:
prompt_base = "Por favor, genera una historia de usuario detallada basada en este issue."
generar_historias("issuesComments.json", "issues_con_historias.json", prompt_base)